In [ ]:
import numpy as np
import time
# Students will submit their files with their team-name.py
# Student have to use the Team as their parent class

In [ ]:
from scipy.optimize import minimize_scalar
class Agent:
  def __init__(self):
    self.time = 1

  def get_action(self, wickets, runs_scored):
    action = self.kl_ucb()
    self.time +=1
    return action

  def KL(self):
    weights = []
    for i in range(6):
      C = (np.log(self.time) + 3*np.log(np.log(self.time)))
      q = minimize_scalar(lambda q: (np.log(1/(1-q)) - C)**2,bounds=(0,1)).x
      weights.append(min(q,1))
    return weights

  def kl_ucb(self):
    if self.time < 7:
      return self.time-1
    UCB_weight = self.KL()
    a_t = np.argmax(UCB_weight)
    return a_t



In [ ]:
class Environment:
  def __init__(self,num_balls,agent):
    self.num_balls = num_balls
    self.agent = agent
    self.__run_time = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__wicket = 0
    self.__end_time = 0
    self.__regret_w = 0
    self.__regret_s = 0
    self.__regret_rho = 0
    self.__p_out =np.array([0.001,0.01,0.02,0.03,0.1,0.3])
    self.__p_run =np.array([1,0.9,0.85,0.8,0.75,0.7])
    self.__action_runs_map = np.array([0,1,2,3,4,6])
    self.__s = (1-self.__p_out)*self.__p_run*self.__action_runs_map
    self.__rho = self.__s/self.__p_out


  def __get_action(self):
    self.__start_time = time.time()
    action  = self.agent.get_action(self.__runs_scored, self.__wicket)
    self.__end_time  = time. time()
    self.__run_time = self.__run_time + self.__end_time - self.__start_time
    return action


  def __get_outcome(self, action):
    pout = self.__p_out[action]
    prun= self.__p_run[action]
    wicket = np.random.choice(2,1,p=[1-pout,pout])[0]
    runs = 0
    if(wicket==0):
      runs = self.__action_runs_map[action]*np.random.choice(2,1,p=[1-prun,prun])[0]
    return wicket, runs


  def innings(self):
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0

    for ball in range(self.num_balls):
      action = self.__get_action()
      self.__wicket, self.__runs_scored   = self.__get_outcome(action)
      self.__total_runs     = self.__total_runs + self.__runs_scored
      self.__total_wickets  = self.__total_wickets + self.__wicket
      self.__regret_w       = self.__regret_w+ (self.__p_out[action]-np.min(self.__p_out))
      self.__regret_s       = self.__regret_s - ((1-self.__p_out[action])*self.__p_run[action]*self.__action_runs_map[action]-np.max(self.__s))
      self.__regret_rho       = self.__regret_rho- ((1-self.__p_out[action])*self.__p_run[action]*self.__action_runs_map[action]/self.__p_out[action]-np.max(self.__rho))
    return self.__regret_w,self.__regret_s,self.__regret_rho, self.__total_runs, self.__total_wickets, self.__run_time


In [ ]:
class Agent2:
  def __init__(self):
    self.no_arms = 6
    self.action_score = np.zeros(self.no_arms)
    self.time = 1
    self.arm_pull = np.ones(self.no_arms)

  def get_action(self,runs_scored, wicket):
    action = self.KL_UCB()
    self.time+=1
    self.weight(action,runs_scored)
    return action

  def weight(self,action ,runs_scored):
    self.action_score[action] = (self.arm_pull[action]*self.action_score[action] + runs_scored)/(self.arm_pull[action]+1)
    self.arm_pull[action] +=1

  def KL_UCB_update(self):
    return self.action_score + np.sqrt(1.5*np.log(self.time)/self.arm_pull)

  def KL_UCB(self):
    if self.time<7:
      return self.time-1
    UCB_score = self.KL_UCB_update()
    action = np.argmax(UCB_score)
    return action

In [ ]:
class Agent3:
  def __init__(self):
      self.actions=np.array([0,1,2,3,4,5])
      self.balls=0
      self.K=len(self.actions)
      self.arm_rewards=np.zeros(self.K)
      self.arm_costs=np.zeros(self.K)
      self.prev_action=0
      self.act_ind=0
      self.iter=1
      self.delta=0.05
      self.rate_estimator=np.zeros(self.K)

  def get_action(self,wicket,runs_scored):

    if self.balls==0:
      self.balls+=1
      return 0

    if len(self.actions)==1:
      self.balls+=1
      print(self.actions[0])
      return self.actions[0]

    if self.act_ind<len(self.actions):
      self.arm_rewards[self.prev_action]+= runs_scored
      self.arm_costs[self.prev_action]+= wicket

      action = self.actions[self.act_ind]
      self.act_ind+=1
      self.prev_action=action

    else:
      self.arm_rewards[self.prev_action]+= runs_scored
      self.arm_costs[self.prev_action]+= wicket

      best_arm=np.argmax([(self.arm_rewards[i]+1e-6)/(self.arm_costs[i]+1e-6) for i in self.actions])
      beta_t= np.sqrt(np.log(4*self.K/self.delta)/2*self.iter)

      for arm in self.actions[self.actions != best_arm]:
        if self.arm_rewards[arm]-2*beta_t<=0:
          continue
        elif ((self.arm_rewards[best_arm]- beta_t)/(self.arm_costs[best_arm]+ beta_t))> ((self.arm_rewards[arm]+ beta_t)/(self.arm_costs[arm]- beta_t)):
          self.actions=self.actions[self.actions!=arm]
      self.act_ind=0
      action=self.actions[self.act_ind]
      self.act_ind+=1
      self.prev_action=action
      self.iter+=1
    print(action)
    return action




In [ ]:
agent = Agent3()
environment = Environment(100,agent)
regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time = environment.innings()

In [ ]:
print(regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time)

7.308000000000007 123.04299999999984 4148.800000000001 169 9 0.054882049560546875
